# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

/usr/lib/python3/dist-packages/matplotlib/__init__.py:874: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, on=['idpostulante', 'nombre', 'estado']\
                       , how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [4]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada una de las combinaciones de título y estado de título, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [5]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
114141,OqrwQqr,Universitario,Abandonado,4.0
161800,MVrWoPL,Universitario,En Curso,4.0
553369,2zQMmd1,Terciario/Técnico,En Curso,3.0
467246,OqrDBYN,Universitario,Graduado,4.0
585318,Np1JmV,Otro,Graduado,1.0


Analizamos la cantidad de IDs duplicados

In [6]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [7]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [8]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
631252,mzP6dk9,Universitario,En Curso,4.0
104150,kPjjK6W,Secundario,Graduado,2.0
405725,owzb3J0,Secundario,Graduado,2.0
241104,5OZkop,Universitario,Graduado,4.0
3849,4rejP1q,Secundario,Graduado,2.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [9]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [10]:
postulantes_educ['estado'].value_counts()

Graduado      253166
En Curso      148362
Abandonado     46381
Name: estado, dtype: int64

In [11]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios,esta_estudiando
559769,vVjNxAx,Terciario/Técnico,En Curso,3.0,1.0
22981,ZDr95v8,Secundario,Graduado,2.0,0.0
258971,LNeKd9p,Secundario,Graduado,2.0,0.0
415579,EzOwwe8,Secundario,Graduado,2.0,0.0
92831,0zN5Oed,Secundario,Graduado,2.0,0.0


Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'orden_estudio' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,estudios,esta_estudiando
619121,eko3kWp,2.0,0.0
183249,1DBLe3,4.0,0.0
181081,X95VGBb,2.0,0.0
344391,Zzw2Qz,4.0,0.0
98832,8EleLD,4.0,0.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
140539,96W1ez8,1993-11-10,MASC
155506,DrpJNZa,1986-01-04,MASC
243114,MVrvBMr,1983-01-12,MASC
236551,MVrpKAv,1977-09-29,FEM
200027,owaZD80,1977-12-02,FEM


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
363905,6rPP8wv,30,MASC
448022,LN8aQzr,48,FEM
227674,JBrqdkk,32,FEM
19204,e4lpK9,37,FEM
174288,JBdJROE,52,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
501524,Bm5b59Z,23,MASC,1.0
60724,131MEO,27,MASC,2.0
35987,6aGWNv,31,MASC,3.0
258446,Oqrl93N,23,FEM,1.0
427179,11rv2L,31,FEM,3.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
338312,3NP9pRJ,MASC,1.0
206959,3N25JPQ,MASC,3.0
185311,dY5BJOq,FEM,1.0
450307,6rBkvEl,FEM,1.0
218478,zvawk2D,MASC,4.0


In [25]:
#asigno a variable numerica el sexo

In [26]:
postulantes_gen_nac["sexo"].value_counts()

FEM           250194
MASC          226713
NO_DECLARA      1094
Name: sexo, dtype: int64

Pasamos a valor numérico el sexo de los postulantes.

- FEM = 1
- MASC = 2
- NO_DECLARA = 0

In [27]:
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'FEM'),'sexo'] = 1

postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'MASC'),'sexo'] = 2

postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'NO_DECLARA'), 'sexo'] = 0

postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
115199,KBzQ8JX,1,2.0
29419,ZGeR4Y,2,3.0
95489,GNJMwNe,1,1.0
256223,zvaNDr3,1,1.0
476194,0zkJeJ8,1,1.0


In [28]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null int64
rango_edad      478001 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


Liberamos memoria para mejorar la performance

In [29]:
postulantes_educ_2=0
postulantes_gen_nac_2=0
fecha_nac=0
año_nac=0
edad_min=0
edad_max=0
condicion_final=0
rango_18_25=0
rango_26_30=0
rango_31_40=0
rango_41_68=0

### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [30]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,a0XaWD,7.0,0.0,2,4.0
1,4reGo5z,7.0,0.0,1,2.0
2,X9lpKkb,7.0,0.0,2,4.0
3,RzMXJ4E,7.0,0.0,1,4.0
4,akOA6b9,7.0,0.0,2,4.0


In [31]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438451 entries, 0 to 438450
Data columns (total 5 columns):
idpostulante       438451 non-null object
estudios           438451 non-null float64
esta_estudiando    438451 non-null float64
sexo               438451 non-null int64
rango_edad         438451 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 20.1+ MB


### DF 4: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [32]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [33]:
len(postulaciones)

6604534

In [34]:
len(postulaciones.drop_duplicates(['idaviso','idpostulante']))

6603752

In [35]:
#como hay solo una diferencia de 800(aprox) duplicados, los tiro
postulaciones.drop_duplicates(['idaviso','idpostulante'], inplace=True)
len(postulaciones)

6603752

In [36]:
#elimino la columna fechapostulacion ya que no nos brinda nada.
postulaciones.drop(columns={'fechapostulacion'}, inplace = True)

In [37]:
postulaciones.sample(1)

,idaviso,idpostulante
5570003,1112464682,pzMDMAp


In [ ]:
#agregar algo piola para el informe, por ej. 
#vemos cual es un estimado al "maximo de postulantes por aviso"

In [38]:
#armo un nuevo df que tenga idaviso | #postulantes
cant_postulantes_por_aviso = postulaciones.groupby(['idaviso',]).size().to_frame('cant_postulaciones').reset_index()
#lo agrupo para tener el top de avisos
cant_postulantes_por_aviso.sort_values('cant_postulaciones', inplace = True, ascending = False)
cant_postulantes_por_aviso

,idaviso,cant_postulaciones
10405,1112334791,24420
1084,1112094756,22424
931,1112033906,20570
10402,1112334788,16687
1640,1112196813,13619
6977,1112280937,12124
447,1111753681,11102
1837,1112204682,10098
11224,1112345900,9784
766,1111970596,9746


In [39]:
cant_postulantes_por_aviso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20231 entries, 10405 to 1874
Data columns (total 2 columns):
idaviso               20231 non-null int64
cant_postulaciones    20231 non-null int64
dtypes: int64(2)
memory usage: 474.2 KB


In [40]:
#agrego una columna de se postulo
postulaciones['sepostulo_x'] = 1

postulaciones.sample(10)

,idaviso,idpostulante,sepostulo_x
3343271,1112340376,3NdNd9X,1
874115,1112320227,eJK5pB,1
2625541,1112340902,bOj3rlb,1
3698829,1112227091,QNPdEoq,1
2645484,1112291495,ZB0Kx1,1
1869884,1112204681,dYoJNka,1
4335553,1112375117,ekO6WG8,1
3153471,1112226788,a0EjqN,1
6139850,1111970596,1R8DLb,1
4502033,1112377206,1Qd2veO,1


### DF 6: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [41]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
3265,1112419549,1,Asesor Comercial con Movilidad Propia Zona Sur...,"<p>Para nuestro cliente, importante Cía. de co...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,Interacción Laboral
7334,1112368795,1,Vendedor de Canal Indirecto adhesión de comerc...,"<p align=""left"" style=""""><span style="""">Para e...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ventas,HuCap
22697,1112500131,1,ANALISTA ADMINISTRATIVO DE RECURSOS HUMANOS,<p>Nos encontramos en la búsqueda de un Analis...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Administración de Personal,ACEROS CORONA
22898,1112503608,1,Encargado de depósito,"<p align=""center"" style=""""><strong><span style...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Almacén / Depósito / Expedición,Desafío Positivo. Gestión en Recursos Humanos
4683,1112358991,1,Técnico de Mantenimiento,<p>Nos orientamos a Técnicos Electromecánicos ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Mantenimiento y Limpieza,FABRICA SRL


In [42]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25563 entries, 0 to 25562
Data columns (total 11 columns):
idaviso                 25563 non-null int64
idpais                  25563 non-null int64
titulo                  25563 non-null object
descripcion             25563 non-null object
nombre_zona             25563 non-null object
ciudad                  137 non-null object
mapacalle               1854 non-null object
tipo_de_trabajo         25563 non-null object
nivel_laboral           25563 non-null object
nombre_area             25563 non-null object
denominacion_empresa    25556 non-null object
dtypes: int64(2), object(9)
memory usage: 2.3+ MB


In [43]:
#elimino las columnas nombre_empresa y descripcion, ya que no vamos a usar esa info
#podriamos eliminar titulo tambien, esperar a si hacemos TF-IDF
avisos_detalles.drop(columns={'descripcion','denominacion_empresa'}, inplace=True)

In [44]:
avisos_detalles.sample(5)

,idaviso,idpais,titulo,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area
14854,1112396569,1,Coordinador Contable,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Contabilidad
6105,1112429841,1,Mecánico de Mantenimiento y Liofilización,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Mantenimiento
22180,1112494217,1,ASISTENTE CONTABLE,Capital Federal,NaN,Vicente Fidel Lopez 70 3A Martínez Provincia d...,Part-time,Senior / Semi-Senior,Contabilidad
14571,1112264277,1,Ejecutivo de Cuentas Semi Senior / Vendedor de...,Capital Federal,NaN,General Lucio Mansilla Piso 10,Full-time,Senior / Semi-Senior,Comercial
11044,1112120346,1,Soldador de Acero Inoxidable - Herrero zona Sa...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Mantenimiento y Limpieza


Eliminamos los avisos duplicados

In [45]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad' y 'mapacalle'.

In [46]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

1    24950
Name: idpais, dtype: int64

In [47]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

True     24818
False      132
Name: ciudad, dtype: int64

In [48]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

True     23152
False     1798
Name: mapacalle, dtype: int64

In [49]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle'}, axis=1, inplace=True)

Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [50]:
avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              22972
Capital Federal                 1914
Buenos Aires (fuera de GBA)       42
La Plata                           4
Ciudad de Mendoza                  3
GBA Oeste                          3
Mendoza                            3
Rosario                            2
Neuquen                            1
Catamarca                          1
Tucuman                            1
Cordoba                            1
San Juan                           1
Santa Fe                           1
Santa Cruz                         1
Name: nombre_zona, dtype: int64

In [51]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

Gran Buenos Aires = 3
GBA Oeste = 3
La Plata = 3
Capital Federal = 2
Buenos Aires (fuera de GBA) = 2
Otros = 1

In [52]:
avisos_detalles['tipo_de_trabajo'].value_counts()

Full-time          22523
Part-time           1725
Teletrabajo          248
Por Horas            124
Pasantia             119
Temporario            89
Por Contrato          87
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [53]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [54]:
avisos_detalles['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16977
Junior                                   4149
Otro                                     1975
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [55]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1

In [56]:
avisos_detalles.sample(5)

,idaviso,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
19811,1111942486,Operario de Producción C/exp en laboratorio - ...,3,10,3,Producción
23961,1111474259,Analista de Generación de Demanda (Negocio Onl...,3,10,3,Marketing
17474,1112273918,Repositor para supermercado,2,10,3,Abastecimiento
4062,1112422210,Atención al cliente telefónica para importante...,3,10,1,Atención al Cliente
20714,1112477253,Ingeniero Mecanico - Cba / San Luis,3,10,3,Ingeniería Mecánica


In [57]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24950 entries, 0 to 25549
Data columns (total 6 columns):
idaviso            24950 non-null int64
titulo             24950 non-null object
nombre_zona        24950 non-null int64
tipo_de_trabajo    24950 non-null int64
nivel_laboral      24950 non-null int64
nombre_area        24950 non-null object
dtypes: int64(4), object(2)
memory usage: 1.3+ MB


Liberamos memoria para mejorar la performance

In [58]:
lista=0
fecha=0
postulaciones_2=0
avisos_detalles_2=0
postulantes_educ=0
postulantes_gen_nac=0

In [59]:
# DEJAR CORRIENDO ESTO TODA LA NOCHE

In [60]:
def devolver_n_ids(n_ids):
    return np.array(postulantes['idpostulante'].sample(n_ids))

In [61]:
t0 = time()

ids_avisos = np.array(avisos_detalles['idaviso'])
n_ids = 250

lista_avisos_postulantes = []

for idaviso in ids_avisos:
    ids_postulantes = devolver_n_ids(n_ids)
    for idpostulante in ids_postulantes:
        tupla = (idaviso, idpostulante)
        lista_avisos_postulantes.append(tupla)
        
tf = time() - t0
print("Tiempo de ejecución:", tf)

Tiempo de ejecución: 0.10625720024108887


In [62]:
df_avisos_postulantes_all = pd.DataFrame(lista_avisos_postulantes, columns=['idaviso','idpostulante'])
df_avisos_postulantes_all['sepostulo_y'] = 0

df_avisos_postulantes_all.head(10)

,idaviso,idpostulante,sepostulo_y
0,1112231885,8kW8MW,0
1,1112231885,e3VOjW,0
2,1112231885,Bma9VJQ,0
3,1112231885,jkjOz4Q,0
4,1112231885,lD2PKXz,0
5,1112231885,5x3bJX,0
6,1112231885,EzZj5PJ,0
7,1112231885,6rLm1AM,0
8,1112231885,VNev56d,0
9,1112231885,bOjXZLd,0


In [63]:
df_avisos_postulantes_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
idaviso         100 non-null int64
idpostulante    100 non-null object
sepostulo_y     100 non-null int64
dtypes: int64(2), object(1)
memory usage: 2.4+ KB


In [64]:
t0 = time()

modelo = pd.merge(df_avisos_postulantes_all, postulaciones, on=['idaviso','idpostulante'],\
                  how='outer')

tf = time() - t0
print("Tiempo de ejecución:", tf)

modelo.head(5)

Tiempo de ejecución: 2.5645406246185303


,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1112231885,8kW8MW,0.0,NaN
1,1112231885,e3VOjW,0.0,NaN
2,1112231885,Bma9VJQ,0.0,NaN
3,1112231885,jkjOz4Q,0.0,NaN
4,1112231885,lD2PKXz,0.0,NaN


In [67]:
modelo.loc[(modelo['sepostulo_x'].isnull()), 'sepostulo_x'] = 0
modelo.loc[(modelo['sepostulo_y'].isnull()), 'sepostulo_y'] = 0
modelo.head(5)

,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1112231885,8kW8MW,0.0,0.0
1,1112231885,e3VOjW,0.0,0.0
2,1112231885,Bma9VJQ,0.0,0.0
3,1112231885,jkjOz4Q,0.0,0.0
4,1112231885,lD2PKXz,0.0,0.0


In [68]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603852 entries, 0 to 6603851
Data columns (total 4 columns):
idaviso         int64
idpostulante    object
sepostulo_y     float64
sepostulo_x     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 251.9+ MB


In [69]:
t0 = time()

modelo['sepostulo'] = modelo['sepostulo_x'] + modelo['sepostulo_y']
modelo.drop(columns={'sepostulo_x','sepostulo_y'}, inplace=True)

tf = time() - t0
print("Tiempo de ejecución:", tf)

modelo.head(5)

/home/towers/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


Tiempo de ejecución: 0.362027645111084


,idaviso,idpostulante,sepostulo
0,1112231885,8kW8MW,0.0
1,1112231885,e3VOjW,0.0
2,1112231885,Bma9VJQ,0.0
3,1112231885,jkjOz4Q,0.0
4,1112231885,lD2PKXz,0.0


In [70]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603852 entries, 0 to 6603851
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
sepostulo       float64
dtypes: float64(1), int64(1), object(1)
memory usage: 201.5+ MB


In [71]:
modelo['sepostulo'].value_counts()

1.0    6603752
0.0        100
Name: sepostulo, dtype: int64

In [73]:
modelo['idaviso'] = modelo['idaviso'].astype('object')
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6603852 entries, 0 to 6603851
Data columns (total 3 columns):
idaviso         object
idpostulante    object
sepostulo       float64
dtypes: float64(1), object(2)
memory usage: 201.5+ MB


In [ ]:
#hacemos 2 merge para agregar informacion sobre los avisos y los postulantes

In [74]:
modelo = pd.merge(modelo, avisos_detalles, on=['idaviso'], how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1112231885,8kW8MW,0.0,Auxiliar Contable,3,10,3,Contabilidad
1,1112231885,e3VOjW,0.0,Auxiliar Contable,3,10,3,Contabilidad
2,1112231885,Bma9VJQ,0.0,Auxiliar Contable,3,10,3,Contabilidad
3,1112231885,jkjOz4Q,0.0,Auxiliar Contable,3,10,3,Contabilidad


In [75]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6040265 entries, 0 to 6040264
Data columns (total 8 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
dtypes: float64(1), int64(3), object(4)
memory usage: 414.8+ MB


In [78]:
#esto es solo para verificar que se mergeo todo piola (True, True)
(modelo['tipo_de_trabajo'].isnull().sum() == 0 , modelo['nivel_laboral'].isnull().sum() == 0)

(True, True)

In [80]:
modelo = pd.merge(modelo, postulantes, on='idpostulante', how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,estudios,esta_estudiando,sexo,rango_edad
0,1112231885,8kW8MW,0.0,Auxiliar Contable,3,10,3,Contabilidad,4.0,0.0,1,3.0
1,1112352890,8kW8MW,1.0,Selector/a Ssr. Freelance,3,10,3,Selección,4.0,0.0,1,3.0
2,1112305942,8kW8MW,1.0,Gerente General,3,10,5,Gerencia / Dirección General,4.0,0.0,1,3.0
3,1112354955,8kW8MW,1.0,Técnico en seguridad e Higiene - Mar Del Plata,3,10,2,Producción,4.0,0.0,1,3.0


In [81]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5792410 entries, 0 to 5792409
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
estudios           float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(4), object(4)
memory usage: 574.5+ MB


In [84]:
#esto es solo para verificar que se mergeo todo piola (True, True)
(modelo['estudios'].isnull().sum() == 0 , modelo['rango_edad'].isnull().sum() == 0)

(True, True)

In [ ]:
#liberamos memoria
df_avisos_postulantes_all=0
postulantes=0
postulaciones=0
avisos_detalles=0

In [ ]:
# ACA TERMINA LA EJECUCION LARGA

In [85]:
modelo.rename(columns={'estudios':'nivel_estudios'},inplace=True)
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5792410 entries, 0 to 5792409
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(4), object(4)
memory usage: 574.5+ MB


Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

In [86]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5792410 entries, 0 to 5792409
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
titulo             object
nombre_zona        int64
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(4), object(4)
memory usage: 574.5+ MB


Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el dataframe.

In [87]:
modelo["sepostulo"].value_counts()

1.0    5792310
0.0        100
Name: sepostulo, dtype: int64

Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones

In [88]:
no = modelo["sepostulo"] == 0
si = modelo["sepostulo"] == 1

modelo_no = modelo[no]
modelo_si = modelo[si]

In [98]:
#puede fallar, depende del random. si falla bajar a 
#n_samples = 250000


n_samples = 500000

modelo_no = modelo_no.sample(n_samples)
modelo_si = modelo_si.sample(n_samples)

modelo_final = pd.concat([modelo_no, modelo_si])

modelo_final.sample(5)

,idaviso,idpostulante,sepostulo,titulo,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nivel_estudios,esta_estudiando,sexo,rango_edad
54699,1112416153,ow5L1Kr,0.0,VENDEDOR/A CON MOVILIDAD,3,10,3,Ventas,3.0,1.0,1,2.0
3125426,1112355348,5md5ADv,1.0,Operarios de Carga y Descarga - Zona San Martin,3,10,2,Producción,2.0,0.0,2,1.0
79864,1112398847,GNo9vvA,0.0,Diseñador Gráfico,3,10,2,Diseño Gráfico,4.0,1.0,1,1.0
998117,1111709950,zv4RbAD,1.0,Gestores Call Center de Cobranza CON EXPERIENCIA,2,9,3,Call Center,3.0,1.0,1,3.0
2120960,1112263367,6KO3px,1.0,Administración - Operaciones,3,10,1,Control de Gestión,4.0,1.0,2,3.0


In [99]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 54690 to 3855034
Data columns (total 12 columns):
idaviso            100 non-null object
idpostulante       100 non-null object
sepostulo          100 non-null float64
titulo             100 non-null object
nombre_zona        100 non-null int64
tipo_de_trabajo    100 non-null int64
nivel_laboral      100 non-null int64
nombre_area        100 non-null object
nivel_estudios     100 non-null float64
esta_estudiando    100 non-null float64
sexo               100 non-null int64
rango_edad         100 non-null float64
dtypes: float64(4), int64(4), object(4)
memory usage: 10.2+ KB


In [101]:
#liberamos memoria
modelo_no=0
modelo_si=0

Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [103]:
modelo_final = modelo_final[['idaviso','idpostulante','rango_edad','sexo',\
                 'nivel_estudios','esta_estudiando','titulo','tipo_de_trabajo',\
                 'nivel_laboral','nombre_zona','nombre_area','sepostulo']]

modelo_final.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,titulo,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo
1746505,1112376879,aBkXkN,3.0,2,4.0,0.0,Estudiantes en Ciencias Económicas con Inglés ...,10,3,3,Impuestos,1.0
2474090,1112400573,5mPV0JX,1.0,1,4.0,1.0,"Encargado/a, Consultoras/es y Cajeros/as - A...",10,3,3,Ventas,1.0
79906,1112398847,vVDP3ve,4.0,2,4.0,0.0,Diseñador Gráfico,10,2,3,Diseño Gráfico,0.0
4318,1111677493,EzeroQo,3.0,1,3.0,0.0,Promotora temporal para exposición - Zona Pilar,10,1,3,Atención al Cliente,0.0
15,1112231885,Bma9VJQ,2.0,1,4.0,1.0,Auxiliar Contable,10,3,3,Contabilidad,0.0


Transformamos las variables categóricas de entero, objeto o flotante a categóricas propiamente dichas

In [105]:
modelo_final['rango_edad'] = modelo_final['rango_edad'].astype('category')
modelo_final['sexo'] = modelo_final['sexo'].astype('category') 
modelo_final['nivel_estudios'] = modelo_final['nivel_estudios'].astype('category')
modelo_final['esta_estudiando'] = modelo_final['esta_estudiando'].astype('category')
modelo_final['tipo_de_trabajo'] = modelo_final['tipo_de_trabajo'].astype('category')
modelo_final['nivel_laboral'] = modelo_final['nivel_laboral'].astype('category')
modelo_final['nombre_zona'] = modelo_final['nombre_zona'].astype('category')
modelo_final['sepostulo'] = modelo_final['sepostulo'].astype('category')

In [106]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 54690 to 3855034
Data columns (total 12 columns):
idaviso            100 non-null object
idpostulante       100 non-null object
rango_edad         100 non-null category
sexo               100 non-null category
nivel_estudios     100 non-null category
esta_estudiando    100 non-null category
titulo             100 non-null object
tipo_de_trabajo    100 non-null category
nivel_laboral      100 non-null category
nombre_zona        100 non-null category
nombre_area        100 non-null object
sepostulo          100 non-null category
dtypes: category(8), object(4)
memory usage: 5.6+ KB


Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [107]:
modelo_final.to_csv("modelo_final.csv", encoding = "utf-8", index = False)